# Web Log Mining for Content Optimization using Apriori Algorithm

##### Web mining process can be defined as the process of applying data mining algorithms or techniques on web data so as to to discover the interesting or access patterns or knowledge to study user behaviour or user access patterns

### Data Used:- NASA web access logs for July and August, 1995.

In [13]:
#Import required libraries

import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import warnings
from apyori import apriori

In [2]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
pd.set_option('float_format', '{:f}'.format)

In [16]:
#Extract the gz files and import dataset provided in .TSV format.
df1 = pd.read_csv(r"19950801.00-19950901.00.tsv", sep="\t", encoding="ISO-8859-1")
df2 = pd.read_csv(r"19950630.23-19950801.00.tsv", sep="\t", encoding="ISO-8859-1")

In [20]:
#combine both the datasets.
df = pd.concat([df1,df2])

###### EDA, Feature Engineering

In [2]:
#check for columns with all null value
df.isnull().values.all(axis=0)

In [121]:
#check for column with any null value
df.isnull().values.any(axis=0)

In [25]:
###### columns 'referer', 'useragent' contain all null values and 'logname' has all '-' values. Hence will be dropped
df = df.loc[:,~df.columns.isin(['referer', 'useragent', 'logname'])]

In [3]:
uniq_host = df.host.nunique()
print("Unique hosts : {}".format(uniq_host))

In [4]:
uniq_url = df.url.nunique()
print("Unique url : {}".format(uniq_url))

In [28]:
#Create timestamp column by converting time variable to datetime format
df['timestamp']= list(map(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'), df['time']))
df=df.drop(columns = 'time')

In [30]:
df['timestamp'] = pd.to_datetime(df.timestamp)
df["date"]=df['timestamp'].dt.strftime("%Y-%m-%d")
df["time"]=df['timestamp'].dt.strftime("%H:%M:%S")

In [31]:
df['day'] = df.timestamp.dt.day
df['month'] = df.timestamp.dt.month
df['weekday']= df.timestamp.dt.day_name()
df['hour']= df.timestamp.dt.hour
df['minute']= df.timestamp.dt.minute
df['second']= df.timestamp.dt.second

In [125]:
#count of rows for each month
month_count = df['month'].value_counts()

##### It was mentioned in the description that data provided is for the month of July and August,1995. However we see that we have data for the month of september as well. 

In [33]:
count_day_month = df.groupby(['month','day'], as_index=False).count()

In [34]:
print('unique days in month 7 : {}'.format(count_day_month[count_day_month['month']==7]['day'].nunique()))
print('unique days in month 8 : {}'.format(count_day_month[count_day_month['month']==8]['day'].nunique()))
print('unique days in month 9 : {}'.format(count_day_month[count_day_month['month']==9]['day'].nunique()))

unique days in month 7 : 29
unique days in month 8 : 31
unique days in month 9 : 1


In [35]:
# As there is just one day data for the month of september, dropping it from the dataset
df = df[df['month']!=9]

In [119]:
#Top 5 types of requested pages
df['ext'] = df['url'].str.split('.').str[-1]
df['ext'] = df['ext'].str.lower()

In [7]:
sns.set(font_scale = 1.5) #set the font of x and y tick labels
df.ext.value_counts()[:5].plot.bar(figsize=(10,5))

###### As per the above graph was can see that there are urls' with different extensions. Removing unwanted files or pages or URLs include removing urls's which are not required for a praticular use case scenario. Here we will keep only the urls' with  extension “.html” and remove records with other extensions like “gif”, “jpeg”, “css” and so on .

###### Records with error status codes are also removed. 

### HTTP Status Messages

###### 200:Successful:OK - The request is OK (this is the standard response for successful HTTP requests)
###### 302:Redirection:Found: - The requested page has moved temporarily to a new URL 
###### 304:Redirection:Not Modified:-Indicates the requested page has not been modified since last requested
###### 403:Client Error:Forbidden:-The request was a legal request, but the server is refusing to respond to it
###### 404:Client Error:Not Found:-The requested page could not be found but may be available again in the future
###### 500:Server Error:-Internal Server Error:-A generic error message, given when no more specific message is suitable
###### 501:Server Error:Not Implemented:-The server either does not recognize the request method, or it lacks the ability to fulfill the request

In [39]:
#Removing unwanted rows
df_cleaned = df[(df['ext']=='html') & (df['response']<400)]

In [9]:
#customers access website mostly for which HTTP method?
df_cleaned ['method'].value_counts().plot(kind='bar')
plt.title("method type")
plt.xlabel("method")
plt.ylabel("count")
plt.legend(['get','post','head'],loc="upper right")
plt.show()

In [10]:
#count the values of different HTTP mentods on clean dataset
df_cleaned['method'].value_counts()

In [11]:
#count the values of different response mentods on clean dataset
df_cleaned.groupby('response').size().plot(kind='bar')
plt.title("Response code count")
plt.show()

###### Most of the requests are getting successful responses

In [46]:
zero_bytes = df1_cleaned[df1_cleaned['bytes']==0]
non_zero_bytes = df1_cleaned[df1['bytes']>0]

In [12]:
#plot response code for zero bytes
zero_bytes.groupby('response').size().plot(kind='bar')
plt.title("Response code count zero_bytes")
plt.show()

##### 0 bytes response may be becuase the response is sent from cache of the browser. 

In [13]:
#plot response code for non-zero bytes
non_zero_bytes.groupby('response').size().plot(kind='bar')
plt.title("Response code count non_zero_bytes")
plt.show()

In [14]:
#Top 10 hosts
top_hosts = pd.DataFrame(df_cleaned.host.value_counts()[:10]).reset_index()
top_hosts.columns = ['host', 'count']
plt.figure(figsize=(15,8))
ax = sns.barplot(x='count', y='host', data=top_hosts)
ax.set_xlabel('count')

In [15]:
#Top 10 accessed URLs'
top_url = pd.DataFrame(df_cleaned.url.value_counts()[:10]).reset_index()
top_url.columns = ['url', 'count']
plt.figure(figsize=(15,8))
ax = sns.barplot(x='count', y='url', data=top_url)
ax.set_xlabel('count')

In [58]:
#count unique daily hosts
unq_daily_hosts= df_cleaned.groupby(['month','day'],as_index=False).agg({"host": 'count', "host":pd.Series.nunique})

In [16]:
plt.figure(figsize=(15, 8))
sns.barplot(x="day", 
            y="host", 
            hue="month", 
            data=unq_daily_hosts)
plt.ylabel("Count", size=14)
plt.xlabel("Day", size=14)
plt.title("Daily unique hosts", size=18)

In [60]:
#count total daily hosts
total_daily_hosts = df_cleaned.groupby(['month','day'], as_index=False)['host'].count()

In [61]:
#Average daily request per host
avg_daily_req_per_host = total_daily_hosts['host']/unq_daily_hosts['host']

In [17]:
sns.set_style("darkgrid")
plt.figure(figsize=(15,6 ))
plt.ylabel("Count", size=14)
plt.xlabel("Day", size=14)
plt.plot(avg_daily_req_per_host)
plt.title("Average Daily Request Per Host", size=18)
plt.show()

In [20]:
def convert_to_id(dataframe):
    
    '''
      Names of hosts' and URLs' are very lengthy. To use them in the algorithm, and for easy interpretetion,
      assigning unique IDs to Host and URL.
      
      Function takes in a dataframe with host and url name and returns complete dataframe with new columsn for ID's 
      along with other columns 
    '''
    dataframe1 = dataframe.assign(host_id=(dataframe['host']).astype('category').cat.codes)
    dataframe2 = dataframe1.assign(url_id=(dataframe['url']).astype('category').cat.codes)
    dataframe2 = dataframe2.astype({"host_id":'category', "url_id":'category'})
    
    return dataframe2   

In [ ]:
df_id = convert_to_id(df_cleaned)

In [70]:
#subset the dataset to select the required columns for algorithm input
df_host_url = df_id[['date','url_id', 'host_id']]

In [89]:
#Pivot the dataset

df_host_url['idx'] = df_host_url.groupby('date').cumcount()
df_host_url_pivot = df_host_url.pivot(index='date',columns='idx')[['url_id']]

In [95]:
records = []
for i in range(0, 60): #no of rows
    records.append([str(df_host_url_pivot.values[i,j]) for j in range(0, 10)]) #Here we are selecting only 10 columns. 

In [106]:
association_rules = apriori(records, min_support=0.05, min_lift=4, min_length=2)
association_results = list(association_rules)

In [108]:
sort_df = pd.DataFrame(columns=('Items','Antecedent','Consequent','Support','Confidence','Lift'))

Support =[]
Confidence = []
Lift = []
Items = []
Antecedent = []
Consequent=[]

for RelationRecord in association_results:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Items.append(RelationRecord.items)
        Antecedent.append(ordered_stat.items_base)
        Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
        Lift.append(ordered_stat.lift)

sort_df['Items'] = list(map(set, Items))                                   
sort_df['Antecedent'] = list(map(set, Antecedent))
sort_df['Consequent'] = list(map(set, Consequent))
sort_df['Support'] = Support
sort_df['Confidence'] = Confidence
sort_df['Lift']= Lift

In [109]:
sort_df.sort_values(by ='Lift', ascending = False, inplace = True)

### Explaination of Support, confidence and lift

###### Support refers to the default popularity of an item and can be calculated by finding number of transactions containing a particular item  divided by total number of transactions
###### Support(B) = (Transactions containing (B))/(Total Transactions)


###### Confidence refers to the likelihood that an item B is also bought if item A is bought. It can be calculated by finding the number of transactions  where A and B are bought together, divided by total number of transactions where A is bought
###### Confidence(A→B) = (Transactions containing both (A and B))/(Transactions containing A)


###### Lift(A -> B) refers to the increase in the ratio of sale of B when A is sold. Lift(A –> B) can be calculated by dividing Confidence(A -> B) divided by Support(B)

###### Lift(A→B) = (Confidence (A→B))/(Support (B))


##### Based on patterns provided in the output, web content can be optimised by checking for content similarity.